## TieDyedGuy's Rover Project Test Notebook

This first cell is just our imports and allowing matplotlib to run inline.

In [1]:
%matplotlib inline
#%matplotlib qt # Choose %matplotlib qt to plot to an interactive window (note it may show up behind your browser)
# Make some of the relevant imports
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc # For saving images as needed
import glob  # For reading in a list of images from a folder

## My Perspective Functions

Here I define the functions that will be used in my process_image.  You can read more about them in each defintion.

In [2]:
#Perspect_Transform takes the source points and transforms them into the destination points. 
#This function was ripped directly from the lectures.
def perspect_transform(img, src, dst):  
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))# keep same size as input image
    return warped

#Rover_Coords_Close_Enough looks at all of the points in relation to the rover and filters out ones that are within
#a certain distance to the Rover.  As you get farther away from the Rover the accuracy drops, so this helps
#with Fidelity

def rover_coords_close_enough(xpix, ypix, max_dist):

    good_pixels = np.sqrt(xpix**2 + ypix**2) < max_dist
    xpix_good = xpix[good_pixels]
    ypix_good = ypix[good_pixels]
    return xpix_good, ypix_good


#Find_Obstacle is the function that looks at the image and finds anything that would be classified as an obstacle
#One thing to note is that it will find rock samples as an obstacle. This is by design because if you do not pick
#up the sample, it IS an obstacle. I got caught on it more times than I would like to admit
def find_obstacle(img, rgb_thresh=(160,160,160)):
    color_select = np.zeros_like(img[:,:,0])
    above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                & (img[:,:,1] > rgb_thresh[1]) \
                & (img[:,:,2] > rgb_thresh[2])
    color_select[np.invert(above_thresh)] = 1
    return color_select


#Find_Rock_Sample is the thresholding function to find the rocks.  I used cv2.inRange instead of the other 
#methods because I was having a hard time getting those to work.
def find_rock_sample(img):
    color_select = np.zeros_like(img[:,:,0])
    #Found Example from Slack of using cv2 instead of trying to figure out method like others finds
    low_thresh = np.array([140,100,0], dtype = "uint8")
    high_thresh = np.array([180,180,40], dtype = "uint8")
    mask_rock = cv2.inRange(img, low_thresh, high_thresh)
    return mask_rock
    


#Find_Navigable finds all navigable terrain. This was ripped straight from the lectures.
# Identify pixels above the threshold
# Threshold of RGB > 160 does a nice job of identifying ground pixels only
def find_navigable(img, rgb_thresh=(160, 160, 160)):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                & (img[:,:,1] > rgb_thresh[1]) \
                & (img[:,:,2] > rgb_thresh[2])
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Return the binary image
    return color_select

#Rover_coords is the function that takes the warped image and moves them to rover coordinates. Ripped right
#from the lecture.
# Define a function to convert to rover-centric coordinates
def rover_coords(binary_img):
    # Identify nonzero pixels
    ypos, xpos = binary_img.nonzero()
    # Calculate pixel positions with reference to the rover position being at the 
    # center bottom of the image.  
    x_pixel = np.absolute(ypos - binary_img.shape[0]).astype(np.float)
    y_pixel = -(xpos - binary_img.shape[0]).astype(np.float)
    return x_pixel, y_pixel

#To_Polar_Coords is a function that takes world space coordinates and makes them into a vector + rotation.
#This is used to figure out which way to turn. Ripped right from the lecture.
# Define a function to convert to radial coords in rover space
def to_polar_coords(x_pixel, y_pixel):
    # Convert (x_pixel, y_pixel) to (distance, angle) 
    # in polar coordinates in rover space
    # Calculate distance to each pixel
    dist = np.sqrt(x_pixel**2 + y_pixel**2)
    # Calculate angle away from vertical for each pixel
    angles = np.arctan2(y_pixel, x_pixel)
    return dist, angles

# Rotate_pix is used to change the Rover roated coordinates to the world rotated ones. Ripped right from the
#lecture.
# Define a function to apply a rotation to pixel positions
def rotate_pix(xpix, ypix, yaw):
    # TODO:
    # Convert yaw to radians
    # Apply a rotation
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
                            
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    # Return the result  
    return xpix_rotated, ypix_rotated

#Translate_pix is ripped right from the lecture, it moves pixels.
# Define a function to perform a translation
def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # TODO:
    # Apply a scaling and a translation
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    # Return the result  
    return xpix_translated, ypix_translated

# Pix_To_world is the combination of the last 2 functions. It takes rover based cordinates and moves it to world
# based ones.  Ripped right from lecture.
# Define a function to apply rotation and translation (and clipping)
# Once you define the two functions above this function should work
def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    # Apply rotation
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    # Apply translation
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    # Perform rotation, translation and clipping all at once
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    # Return the result
    return x_pix_world, y_pix_world


## Read in saved data and ground truth map of the world

The next cell builds the base of the test. It reads in sample camera views and also the ground_truth_map, the reality of what we are dealing with.  This is just more setup.

In [3]:
# Import pandas and read in csv file as a dataframe
import pandas as pd
# Change this path to your data directory
df = pd.read_csv('../test_dataset/robot_log.csv', delimiter=';', decimal='.')
csv_img_list = df["Path"].tolist() # Create list of image pathnames
# Read in ground truth map and create a 3-channel image with it
ground_truth = mpimg.imread('../calibration_images/map_bw.png')
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

# Creating a class to be the data container
# Will read in saved data from csv file and populate this object
# Worldmap is instantiated as 200 x 200 grids corresponding 
# to a 200m x 200m space (same size as the ground truth map: 200 x 200 pixels)
# This encompasses the full range of output position values in x and y from the sim
class Databucket():
    def __init__(self):
        self.images = csv_img_list  
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw = df["Yaw"].values
        self.count = -1 # This will be a running index, setting to -1 is a hack
                        # because moviepy (below) seems to run one extra iteration
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d # Ground truth worldmap

# Instantiate a Databucket().. this will be a global variable/object
# that you can refer to in the process_image() function below
data = Databucket()


## TieDyedGuy's Process_Image

Here is my process_image right from my actual Rover.  I mostly skipped this notebook at first and jumped right to working on the Rover.  Then once I had it working there, I came back to this.  I mostly thing I did this because of my inexperience with numpy.  This page was very math / calculation based and I was struggling. When I got to actually move the rover it made more sense.

In [4]:
#Proccess_image: Here we go!
def process_image(img):
    #Setting up our output image. It will be a 2x3 block of images.
    output_image= np.zeros((400,960,3)).astype(np.uint8)
    
    #This is half of what we want the warped world size to be. By setting this to 5, we are saying that 1m^2
    #is going to be a 10x10px square.
    dst_size = 5 
    
    # Set a bottom offset to account for the fact that the bottom of the image 
    # is not the position of the rover but a bit in front of it
    bottom_offset = 6
    
    #This source and destination was ripped right from the lecture.
    source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
    destination = np.float32([[img.shape[1]/2 - dst_size, img.shape[0] - bottom_offset],
                  [img.shape[1]/2 + dst_size, img.shape[0] - bottom_offset],
                  [img.shape[1]/2 + dst_size, img.shape[0] - 2*dst_size - bottom_offset], 
                  [img.shape[1]/2 - dst_size, img.shape[0] - 2*dst_size - bottom_offset],
                  ])

    #Here we find our navigable terrain by warping then thresholding it.This is like the lecture.
    warped = perspect_transform(img, source, destination)
    navigable = find_navigable(warped)
    
    #For Obstacle and Rock_Sample, I run the threshold before I warp the image.  This seemed to work out better
    #but I believe it shouldn't matter.
    obstacle_pre_warp = find_obstacle(img)
    obstacle = perspect_transform(obstacle_pre_warp, source, destination)
    
    #Rock thresholding
    rock_sample_pre_warp = find_rock_sample(img)
    rock_sample = perspect_transform(rock_sample_pre_warp, source, destination)
    
   
    #Building the final squares fo rthe video
    output_image[0:160,0:320,:] = img
    output_image[160:320,0:320,0] = navigable * 255
    output_image[160:320,0:320,1] = navigable * 255
    output_image[160:320,0:320,2] = navigable * 255
    output_image[0:160,320:640,0] = obstacle * 255
    output_image[0:160,320:640,1] = obstacle * 255
    output_image[0:160,320:640,2] = obstacle * 255
    output_image[160:320,320:640,0] = rock_sample * 255
    output_image[160:320,320:640,1] = rock_sample * 255
    output_image[160:320,320:640,2] = rock_sample * 255
   
    #Show what it looks like from the Rover's perspective (This is pre-finding close values)
    rover_vision_image = np.zeros((160,320,3)).astype(np.uint8)
    rover_vision_image[:,:,0] = obstacle * 255
    rover_vision_image[:,:,1] = rock_sample * 255
    rover_vision_image[:,:,2] = navigable * 255
    
    #Convert map image pixel values to rover-centric coords
    xpix_ob, ypix_ob = rover_coords(obstacle)
    xpix_rs, ypix_rs = rover_coords(rock_sample)
    xpix_nav, ypix_nav = rover_coords(navigable)

    #Limit the distance from the Rover to only coordinates we deem close enough. Here we mark it as 40 pixels away
    find_max_distance = 40
    xpix_ob_close, ypix_ob_close = rover_coords_close_enough(xpix_ob, ypix_ob, find_max_distance)
    xpix_rs_close, ypix_rs_close = rover_coords_close_enough(xpix_rs, ypix_rs, find_max_distance)
    xpix_nav_close, ypix_nav_close = rover_coords_close_enough(xpix_nav, ypix_nav, find_max_distance)
   
    #Convert those good pixels to world pixels for the world map.
    ypix_ob_world, xpix_ob_world = pix_to_world(xpix_ob_close, ypix_ob_close, data.xpos[data.count], data.ypos[data.count], data.yaw[data.count], 200, 10)
    ypix_rs_world, xpix_rs_world = pix_to_world(xpix_rs_close, ypix_rs_close, data.xpos[data.count], data.ypos[data.count], data.yaw[data.count], 200, 10)
    ypix_nav_world, xpix_nav_world = pix_to_world(xpix_nav_close, ypix_nav_close, data.xpos[data.count], data.ypos[data.count], data.yaw[data.count], 200, 10)

    
    #Update the worldmap to show what we have found.
    data.worldmap[xpix_ob_world, ypix_ob_world, 0] = 255    
    data.worldmap[xpix_rs_world, ypix_rs_world, 1] = 255
    data.worldmap[xpix_nav_world, ypix_nav_world, 2] = 255
    
    #Creating the world map vs the truth map
    map_add = cv2.addWeighted(data.worldmap, 1, data.ground_truth, 0.5, 0)
    
    #Add the Rover vision and the world map to the movie
    output_image[0:160,640:960,:] = rover_vision_image
    output_image[200:400,640:840,:] = np.flipud(map_add)
    
    if (len(xpix_rs_close)>1):
        cv2.putText(output_image,"Here ->", (380, 300), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)    
    
    #Putting text over the images
    cv2.putText(output_image,"Rover Feed!", (20, 140), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (0, 0, 0), 1)
    cv2.putText(output_image,"Obstacles Threshold!", (340, 140), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(output_image,"Rover Final Vision!", (640, 140), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(output_image,"Navigable Treshold!", (20, 380), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(output_image,"Sample Threshold!", (340, 380), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(output_image,"Final World Map!", (640, 380), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)    
    
    
    
    # Keep track of the index in the Databucket()
    data.count += 1
    
    
    
    return output_image

## Make a video from processed image data
Use the [moviepy](https://zulko.github.io/moviepy/) library to process images and create a video.
  

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip


# Define pathname to save the output video
output = '../output/test_mapping.mp4'
data = Databucket() # Re-initialize data in case you're running this cell multiple times
clip = ImageSequenceClip(data.images, fps=60) # Note: output video will be sped up because 
                                          # recording rate in simulator is fps=25
new_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time new_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video ../output/test_mapping.mp4
[MoviePy] Writing video ../output/test_mapping.mp4


100%|██████████| 452/452 [00:05<00:00, 82.93it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output/test_mapping.mp4 

CPU times: user 5.25 s, sys: 152 ms, total: 5.4 s
Wall time: 5.57 s


### This next cell should function as an inline video player
This normally works for me


In [6]:

from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Below is an alternative way to create a video in case the above cell did not work.

In [7]:
import io
import base64
video = io.open(output, 'r+b').read()
encoded_video = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded_video.decode('ascii')))